# VM

In [ ]:
import numpy as np

import tvm
from tvm.ir import IRModule
from tvm.runtime import vm as _vm
from tvm.relay import vm as rly_vm
from tvm import relay

from tvm.relay import transform
from tvm.relay.prelude import Prelude
from tvm.contrib import utils
from tvm.relay import testing


def create_exec(f, target="llvm", params=None):
    if isinstance(f, relay.Expr):
        mod = IRModule()
        mod["main"] = f
        executable = rly_vm.compile(mod, target=target, params=params)
        return executable
    else:
        assert isinstance(f, IRModule), "expected mod as tvm.IRModule"
        executable = rly_vm.compile(f, target=target, params=params)
        return executable


def get_serialized_output(mod, *data, params=None, target="llvm", device=tvm.cpu()):
    exe = create_exec(mod, target, params=params)
    code, lib = exe.save()
    des_exec = _vm.Executable.load_exec(code, lib)
    des_vm = _vm.VirtualMachine(des_exec, device)
    result = des_vm.run(*data)
    return result

def run_network(mod, params, dtype="float32"):
    def get_vm_output(mod, data, params, target, device, dtype="float32"):
        result = relay.create_executor("vm", mod=mod, device=device).evaluate()(data, **params)
        return result.numpy().astype(dtype)

    data_shape = [int(x) for x in mod["main"].checked_type.arg_types[0].shape]
    data = np.random.uniform(size=data_shape).astype(dtype)
    target = "llvm"
    dev = tvm.cpu(0)

    tvm_out = get_vm_output(mod, tvm.nd.array(data.astype(dtype)), params, target, dev, dtype)
    vm_out = get_serialized_output(
        mod, tvm.nd.array(data.astype(dtype)), params=params, target=target, device=dev
    )
    tvm.testing.assert_allclose(vm_out.numpy().astype(dtype), tvm_out, rtol=1e-5, atol=1e-5)

In [13]:
mod = IRModule()
a = relay.const(1.0, "float32")
x = relay.var("x", shape=(10, 10), dtype="float32")
f1 = relay.Function([x], x + a)
glb_f1 = relay.GlobalVar("f1")
mod[glb_f1] = f1

# TODO(@jroesch): look into optimizing away the need to do this
mod = transform.InferType()(mod)

b = relay.const(2.0, "float32")
y = relay.var("y", shape=(10, 10), dtype="float32")
f2 = relay.Function([y], y - b)
glb_f2 = relay.GlobalVar("f2")
mod[glb_f2] = f2

# TODO(@jroesch): look into optimizing away the need to do this
mod = transform.InferType()(mod)

x1 = relay.var("x1", shape=(10, 10), dtype="float32")
y1 = relay.var("y1", shape=(10, 10), dtype="float32")
main = relay.Function([x1, y1], glb_f1(x1) * glb_f2(y1))
mod["main"] = main

exe = create_exec(mod)
glbs = exe.globals
prim_ops = exe.primitive_ops
code = exe.bytecode

In [10]:
glbs

['f1', 'main', 'f2']

In [12]:
prim_ops

['vm_mod_fused_multiply', 'vm_mod_fused_subtract', 'vm_mod_fused_add']

In [16]:
code, lib = exe.save()

In [18]:
lib

Module(llvm, 561537a0a558)